In [ ]:
# Install dependencies.
!rm -r sample_data
!pip install -qq --upgrade transformers compel accelerate diffusers
from IPython.display import clear_output
clear_output()

In [ ]:
# Set the details for your model here:
import torch

from diffusers import AutoencoderKL, StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
base = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    add_watermarker=False
).to("cuda")

clear_output()

In [ ]:
@torch.no_grad()
def decode_latents(vae, latents, pipe):
    print(vae.config.scaling_factor)
    latents = latents / vae.config.scaling_factor
    image = vae.decode(latents).sample[0]
    image = image.mul(0.5).add(0.5).clamp(0, 1)
    image = image.cpu().permute(1, 2, 0).numpy()
    return pipe.numpy_to_pil(image)

negative_prompt = "bokeh,blur,blurry,blurred image,painting,artwork,blocky,ugly,old,boring,photoshopped,tired,wrinkles,scar,gray hair,big forehead,crosseyed,dumb,stupid,cockeyed,disfigured,assymetrical,unrealistic,grayscale,bad anatomy,unnatural irises,no pupils,blurry eyes,dark eyes,extra limbs,deformed,disfigured eyes,out of frame,no irises,assymetrical face,broken fingers,extra fingers,disfigured hands"
prompt = "close up Photograph of woman in red dress in a luxury garden surrounded with blue,yellow,purple and flowers in many colors,high class,award-winning photography,Portra 400,full format"

latents = base(prompt, negative_prompt=negative_prompt, num_inference_steps=20, width=1280, height=832, guidance_scale = 8, output_type="latent").images

display(decode_latents(base.vae, latents, base)[0])